# Scop3P

A comprehensive database of human phosphosites within their full context. Scop3P integrates sequences (UniProtKB/Swiss-Prot), structures (PDB), and uniformly reprocessed phosphoproteomics data (PRIDE) to annotate all known human phosphosites. 

Scop3P, available at https://iomics.ugent.be/scop3p, presents a unique resource for visualization and analysis of phosphosites and for understanding of phosphosite structure–function relationships.

Please cite: https://doi.org/10.1021/acs.jproteome.0c00306

In [62]:
import requests, tempfile,json
import pandas as pd 
from b2bTools import SingleSeq, constants
import py3Dmol
import os
import nglview as nv


In [63]:
import tempfile

state = {
    "acc": None,
    "ptm_results": None,      # keep if you use it
    "ptm_json": None,         # add from state
    "ptm_table": None,
    "variants": None,
    "variants_df": None,      # add from state (or choose one naming)
    "sequence": None,
    "bio2byte_raw": None,
    "dynamic_properties": None,
    "rin_pdb_path": None,
    "af_path": None,
    "rin_html": None,
    "workdir": tempfile.mkdtemp(prefix="scop3p_session_")
}


In [64]:
def fetch_protein_modifications(accession):
    """
    Fetches protein modifications for a given UniProt ID.

    Parameters:
    accession (str): UniProt ID of the protein.

    Returns:
    dict: A dictionary containing protein modifications.
    """
    BASE_URL = "https://iomics.ugent.be/scop3p/api/modifications"
    url = f'{BASE_URL}?accession={accession}'
    headers = {'accept': 'application/json'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [65]:
def get_modification_table(modifications):
    """
    Displays the protein modifications in a pandas DataFrame.

    Parameters:
    modifications (list): A list of dictionaries, each representing a protein modification.
    """
    df = pd.DataFrame(modifications)
    df = df[['residue', 'name', 'evidence', 'position', 'source', 'reference', 'functionalScore', 'specificSinglyPhosphorylated']]
    
    return df 
    

In [66]:
import requests
import pandas as pd

def fetch_uniprot_variants_disease(accession: str) -> pd.DataFrame:
    """Fetch UniProt (EBI proteins API) variants with disease association for a UniProt accession."""
    url = f"https://www.ebi.ac.uk/proteins/api/variation/{accession}"
    headers = {"Accept": "application/json"}
    r = requests.get(url, headers=headers, timeout=60)
    r.raise_for_status()
    data = r.json()

    rows = []
    for feat in data.get("features", []):
        if feat.get("type") != "VARIANT":
            continue
        for assoc in feat.get("association", []):
            if assoc.get("disease") is not True:
                continue
            begin = feat.get("begin")
            try:
                pos = int(begin) if begin is not None else None
            except Exception:
                pos = None
            rows.append({
                "ACC_ID": accession,
                "position": pos,
                "WT": feat.get("wildType"),
                "MT": feat.get("mutatedType"),
                "consequence": feat.get("consequenceType"),
                "disease_name": assoc.get("name"),
            })

    return pd.DataFrame(rows)


In [67]:
import py3Dmol

def display_local_pdb_3D(modification_table, accession):
    view = py3Dmol.view(width=700, height=500)
    view.addModel(open(accession + '.pdb', 'r').read(), 'pdb')

    view.setStyle({}, {'cartoon': {'color': 'silver'}})
    view.addSurface(py3Dmol.VDW, {'opacity': 0.35, 'color': 'white'}, {})

    # --- Color phosphosites 
    for _, row in modification_table.iterrows():
        position = str(row['position'])

        # Normalize residue label to avoid mismatches
        residue = str(row['residue']).strip()  # removes trailing spaces etc.

        if residue == 'TYR':
            color = '#2CA02C'
        elif residue == 'SER':
            color = '#1F77B4'
        elif residue == 'THR':
            color = '#FF7F0E'
        else:
            color = '#7B241C'

        sel = {'resi': position}  # add {'chain': row['chain']} if needed

        view.addStyle(sel, {'stick': {'color': color}})
        view.addStyle(sel, {'sphere': {'color': color, 'radius': 0.9}})

    # --- Hover for ALL amino acids (all atoms) ---
    view.setHoverable(
        {}, True,
        """
        function(atom, viewer, event, container) {
            if(!atom.label) {
                atom.label = viewer.addLabel(
                    atom.resn + " " + atom.resi + (atom.chain ? (" : " + atom.chain) : ""),
                    {position: atom, backgroundColor: 'mintcream', fontColor: 'black'}
                );
            }
        }
        """,
        """
        function(atom, viewer) {
            if(atom.label) {
                viewer.removeLabel(atom.label);
                delete atom.label;
            }
        }
        """
    )

    view.zoomTo()
    view.render()
    return view


In [68]:
def display_ptm_3D(modification_table, pdb_id, chain=None):
    view = py3Dmol.view(query=f"pdb:{pdb_id}")

    # Protein context
    view.setStyle({}, {'cartoon': {'color': 'skyblue'}})

    # Global surface (NO hover expected here)
    view.addSurface(py3Dmol.VDW, {'opacity': 0.6, 'color': 'white'}, {})

    # ---- Colored modified residues (ATOMS) ----
    for _, row in modification_table.iterrows():
        position = str(row['position'])
        residue  = str(row['residue']).strip()

        if residue == 'TYR':
            color = '#2CA02C'
        elif residue == 'SER':
            color = '#1F77B4'
        elif residue == 'THR':
            color = '#FF7F0E'
        else:
            color = '#7B241C'

        sel = {'resi': position}
        if chain:
            sel['chain'] = chain

        # ATOMS → hover works
        view.addStyle(sel, {'stick':  {'color': color}})
        view.addStyle(sel, {'sphere': {'color': color, 'radius': 0.9}})

    # ---- Hover for ALL amino acids ----
    view.setHoverable(
        {}, True,
        """
        function(atom, viewer, event, container) {
            if (!atom.label) {
                atom.label = viewer.addLabel(
                    atom.resn + " " + atom.resi + (atom.chain ? (" : " + atom.chain) : ""),
                    {position: atom, backgroundColor: 'mintcream', fontColor: 'black'}
                );
            }
        }
        """,
        """
        function(atom, viewer) {
            if (atom.label) {
                viewer.removeLabel(atom.label);
                delete atom.label;
            }
        }
        """
    )

    view.zoomTo()
    view.render()
    return view


In [69]:
import os
import requests

def download_alphafold_pdb(uniprot_acc: str, outdir: str) -> str:
    """
    Downloads AlphaFold DB PDB for a UniProt accession.
    Returns local file path.
    """
    os.makedirs(outdir, exist_ok=True)
    # AFDB file naming convention
    url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_acc}-F1-model_v6.pdb"
    out_path = os.path.join(outdir, f"AF-{uniprot_acc}-F1-model_v6.pdb")

    r = requests.get(url, timeout=60)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        f.write(r.content)

    return out_path


In [70]:
def fetch_sequence_aminoacids(accession):
    BASE_URL = f"http://uniprot.org/uniprotkb/{accession}.fasta"
    url = f'{BASE_URL}?accession={accession}'
    response = requests.get(url)
    if response.status_code == 200:
        raw_fasta_sequence = response.content.decode("utf-8")
    else:
        raw_fasta_sequence = ""
    
    lines = raw_fasta_sequence.split('\n')
    protein_id = str(lines[0])
    amino_acids = "".join([str(l) for l in lines[1:]])
    
    return protein_id, amino_acids

In [71]:
def predict_biophysical_features(accession, sequence):

    with tempfile.NamedTemporaryFile(prefix="seq_", suffix=".fasta", mode="w") as fp:
        fp.write(f">{accession}\n{sequence}\n")
        fp.flush()
        fp.seek(0)
        
        pred = SingleSeq(fp.name).predict(tools=[constants.TOOL_DYNAMINE, constants.TOOL_DISOMINE, constants.TOOL_EFOLDMINE]).get_all_predictions()
    
    return pred


In [72]:
import colorsys


def pseudocolor(minval, maxval,val):
    """ Convert predicted values min.....max in range Green...Yellow..RED 
        The colors correspond to Red and Green in the HSV colorspace
    """
    minval,maxval=float(minval),float(maxval)
    h = (float(maxval-val) / (maxval-minval)) * 120
    r, g, b = colorsys.hsv_to_rgb(h/360, 1., 1.)
    rgb=map(lambda x: int(255 * x), (r, g, b))
    rgb=tuple(rgb)
    rgb='0x%02x%02x%02x' % rgb
    return rgb

In [73]:
def remap(df):
    BDcolor,EFcolor,DOcolor={},{},{}
    seqpos=0
    min_BD,max_BD=min(df.backbone),max(df.backbone)
    min_DO,max_DO=min(df.disoMine),max(df.disoMine)
    min_EF,max_EF=min(df.earlyFolding),max(df.earlyFolding)
    
    for index, row in df.iterrows():
        seqpos+=1
        BDrescol=pseudocolor(min_BD,max_BD,float(row.backbone))
        DOrescol=pseudocolor(min_EF,max_EF,float(row.disoMine))
        EFrescol=pseudocolor(min_EF,max_EF,float(row.earlyFolding))
        BDcolor[seqpos]=BDrescol
        DOcolor[seqpos]=DOrescol
        EFcolor[seqpos]=EFrescol
        
    return BDcolor,EFcolor,DOcolor
        
        

In [74]:
def display_b2b_3D(dynamic_properties, pdb_path: str):
    BDcolor, EFcolor, DOcolor = remap(dynamic_properties)
    modpos = modification_table.position.tolist()

    view = py3Dmol.view(viewergrid=(2,2))
    with open(pdb_path, "r") as f:
        view.addModel(f.read(), "pdb")

    # IMPORTANT: setStyle(selection, style)
    view.setStyle({}, {'cartoon': {'colorscheme': {'prop':'b','gradient':'rwb','min':0.0,'max':100.0}}}, viewer=(0,0))
    view.setStyle({}, {'cartoon': {'colorscheme': {'prop':'resi','map':BDcolor}}}, viewer=(0,1))
    view.setStyle({}, {'cartoon': {'colorscheme': {'prop':'resi','map':DOcolor}}}, viewer=(1,0))
    view.setStyle({}, {'cartoon': {'colorscheme': {'prop':'resi','map':EFcolor}}}, viewer=(1,1))

    # Surface highlight + pickable overlay on mod residues
    for mod in modpos:
        m = str(mod)
        sel = {'resi': m}

        view.addSurface(py3Dmol.VDW, {'opacity': 1.0}, sel, viewer=(0,0))
        view.addSurface(py3Dmol.VDW, {'opacity': 1.0, 'color': BDcolor[mod]}, sel, viewer=(0,1))
        view.addSurface(py3Dmol.VDW, {'opacity': 1.0, 'color': DOcolor[mod]}, sel, viewer=(1,0))
        view.addSurface(py3Dmol.VDW, {'opacity': 1.0, 'color': EFcolor[mod]}, sel, viewer=(1,1))

        # MAKE IT PICKABLE: opacity must be > 0
        for panel in [(0,0), (0,1), (1,0), (1,1)]:
            view.addStyle(sel, {'sphere': {'radius': 0.8, 'opacity': 0.15}}, viewer=panel)
            # optional: stick helps pickability even more
            # view.addStyle(sel, {'stick': {'opacity': 0.15}}, viewer=panel)

    # Background + hover everywhere (per panel)
    for panel in [(0,0), (0,1), (1,0), (1,1)]:
        view.setBackgroundColor('white', viewer=panel)

        view.setHoverable(
            {},  # hover everywhere
            True,
            """
            function(atom, viewer, event, container) {
                if (!atom.label) {
                    atom.label = viewer.addLabel(
                        atom.resn + " " + atom.resi + (atom.chain ? (" : " + atom.chain) : ""),
                        {position: atom, backgroundColor: 'mintcream', fontColor:'black'}
                    );
                }
            }
            """,
            """
            function(atom, viewer) {
                if (atom.label) {
                    viewer.removeLabel(atom.label);
                    delete atom.label;
                }
            }
            """,
            viewer=panel
        )

    view.zoomTo()
    view.render()
    return view


In [75]:
import numpy as np
import networkx as nx
from scipy.spatial import KDTree
from Bio.PDB import PDBParser

def build_geometry_graph_from_pdb(pdb_path, chain="A", cutoff=8.0, atom_name="CA"):
    """
    Build a residue interaction network from a PDB file using CA (or CB fallback) distances.
    Nodes: residue positions (ints)
    Edges: if distance <= cutoff, with attributes distance, weight=1/distance, resistance=distance
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("af", pdb_path)

    # Use first model
    model = next(structure.get_models())

    # Pick chain (AlphaFold is usually 'A')
    if chain not in model:
        chain_obj = next(model.get_chains())
        chain = chain_obj.id  # fallback
    else:
        chain_obj = model[chain]


    coords = []
    meta = []

    for res in chain_obj:
        # standard residues only
        if res.id[0] != " ":
            continue

        resi = int(res.id[1])
        resn = res.resname

        # choose atom
        atom = None
        if atom_name in res:
            atom = res[atom_name]
        elif atom_name == "CB" and "CA" in res:
            atom = res["CA"]
        elif atom_name == "CA":
            # CA required; skip if missing
            continue
        else:
            # fallback to CA if present
            atom = res["CA"] if "CA" in res else None

        if atom is None:
            continue

        coords.append(atom.coord.astype(float))
        meta.append({"Chain": chain, "Residue": resi, "ResName": resn})

    coords = np.asarray(coords, dtype=float)
    if len(coords) == 0:
        raise ValueError("No residue coordinates found. Check chain/atom_name.")

    nodes = [(m["Chain"], int(m["Residue"])) for m in meta]
    tree = KDTree(coords)

    G = nx.Graph(layer=f"geometry:{atom_name}_cut{cutoff}", chain=chain, pdb=pdb_path)

    for n, m in zip(nodes, meta):
        G.add_node(n, **m)

    for i in range(len(nodes)):
        idxs = tree.query_ball_point(coords[i], cutoff)
        for j in idxs:
            if j <= i:
                continue
            d = float(np.linalg.norm(coords[i] - coords[j]))
            w = 1.0 / max(d, 1e-6)
            G.add_edge(nodes[i], nodes[j], weight=w, distance=d, resistance=1.0 / max(w, 1e-9))

    return G, meta


In [76]:
from pyvis.network import Network

def nx_rin_to_pyvis_default(
    G,
    ptm_positions=None,
    mutation_positions=None,
    out_html="rin_pyvis.html",
    height="600px",
    width="100%",
    default_color="#B0B0B0",   # light grey
    ptm_color="#1f77b4",       # blue
    mut_color="#d62728",       # red
    both_color="#9467bd",      # purple
    node_size=30,
    ptm_size=35,
    mut_size=35,
    both_size=40,
    select_menu=True,
    filter_menu=False
):
    ptm_set = set(int(x) for x in (ptm_positions or []))
    mut_set = set(int(x) for x in (mutation_positions or []))

    net = Network(
        height=height,
        width=width,
        directed=False,
        notebook=True,
        cdn_resources="in_line",
        select_menu=select_menu,
        filter_menu=filter_menu
    )

    net.set_options("""
    {
      "groups": {
        "PTM": {
          "color": {
            "background": "#d62728",
            "border": "#d62728",
            "highlight": { "background": "#d62728", "border": "#d62728" },
            "hover":     { "background": "#d62728", "border": "#d62728" }
          }
        },
        "Mutation": {
          "color": {
            "background": "#1f77b4",
            "border": "#1f77b4",
            "highlight": { "background": "#1f77b4", "border": "#1f77b4" },
            "hover":     { "background": "#1f77b4", "border": "#1f77b4" }
          }
        },
        "PTM+Mutation": {
          "color": {
            "background": "#2ca02c",
            "border": "#2ca02c",
            "highlight": { "background": "#2ca02c", "border": "#2ca02c" },
            "hover":     { "background": "#2ca02c", "border": "#2ca02c" }
          }
        },
        "Other": {
          "color": {
            "background": "#9FA8B0",
            "border": "#9FA8B0",
            "highlight": { "background": "#9FA8B0", "border": "#9FA8B0" },
            "hover":     { "background": "#9FA8B0", "border": "#9FA8B0" }
          }
        }
      }
    }
    """)




    # ---- Nodes ----
    for (ch, resi), attrs in G.nodes(data=True):
        resi = int(resi)
        resn = attrs.get("ResName", "")

        is_ptm = resi in ptm_set
        is_mut = resi in mut_set

        if is_ptm and is_mut:
            bg = both_color
            size = both_size
            group = "PTM+Mutation"
        elif is_mut:
            bg = mut_color
            size = mut_size
            group = "Mutation"
        elif is_ptm:
            bg = ptm_color
            size = ptm_size
            group = "PTM"
        else:
            bg = default_color
            size = node_size
            group = "Other"

        node_id = f"{ch}:{resi}"

        net.add_node(
            node_id,
            label=f"{resn} {resi}" if resn else str(resi),
            title=f"Chain: {ch}<br>Residue: {resn}<br>Position: {resi}<br>Group: {group}",
            color={
                "background": bg,
                "border": "#333333",
                "highlight": {"background": bg, "border": "#000000"},
                "hover": {"background": bg, "border": "#000000"}
            },
            size=size,
            group=group,
            font={"size": 12}
        )

    # ---- Edges ----
    for (ch1, r1), (ch2, r2), eattrs in G.edges(data=True):
        a = f"{ch1}:{int(r1)}"
        b = f"{ch2}:{int(r2)}"
        dist = eattrs.get("distance", None)

        net.add_edge(
            a, b,
            color="#A9A9A9",
            title=f"distance: {dist:.2f} Å" if dist is not None else ""
        )

    html = net.generate_html()
    with open(out_html, "w", encoding="utf-8") as f:
        f.write(html)
    
    return out_html


In [77]:
import os
import subprocess
import tempfile
from Bio.PDB import PDBParser, PDBIO, Select
import nglview as nv
import ipywidgets as widgets
from IPython.display import display, clear_output

def save_upload(upload_widget, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    v = upload_widget.value
    if not v:
        raise ValueError("No file uploaded")

    # Newer ipywidgets: tuple/list of dicts
    if isinstance(v, (tuple, list)):
        fileinfo = v[0]
        name = fileinfo.get("name", "upload.pdb")
        content = fileinfo["content"]

    # Older ipywidgets: dict name -> fileinfo
    elif isinstance(v, dict):
        name, fileinfo = next(iter(v.items()))
        content = fileinfo["content"]

    else:
        raise TypeError(f"Unexpected upload_widget.value type: {type(v)}")

    path = os.path.join(out_dir, name)
    with open(path, "wb") as f:
        f.write(content)
    return path



def chain_range_from_pdb(pdb_path, chain_id):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("X", pdb_path)
    residues = [
        res.id[1]
        for model in structure
        for chain in model
        if chain.id == chain_id
        for res in chain
        if res.id[0] == " "
    ]
    if not residues:
        raise ValueError(f"No residues found for chain {chain_id}")
    return min(residues), max(residues)


class ChainRangeSelect(Select):
    def __init__(self, chain_id, start, end):
        self.chain_id = chain_id
        self.start = start
        self.end = end

    def accept_chain(self, chain):
        return chain.id == self.chain_id

    def accept_residue(self, residue):
        r = residue.id[1]
        return (self.start <= r <= self.end)

def run_tmalign_write(pdb1, pdb2, out_dir, out_name):
    os.makedirs(out_dir, exist_ok=True)
    cmd = ["TM-align", os.path.abspath(pdb1), os.path.abspath(pdb2), "-o", out_name]
    res = subprocess.run(cmd, cwd=out_dir, capture_output=True, text=True, check=True)

    candidates = [
        os.path.join(out_dir, out_name),
        os.path.join(out_dir, out_name + ".pdb"),
        os.path.join(out_dir, "TM_sup.pdb"),
    ]
    out_pdb = next((c for c in candidates if os.path.exists(c)), None)
    if out_pdb is None:
        raise RuntimeError(f"No TM-align output found. Files: {os.listdir(out_dir)}")
    return out_pdb, res.stdout

import nglview as nv

def visualize_ngl(pdb_ref, pdb_aligned, selection="protein"):
    view = nv.NGLWidget()

    # CRITICAL: disable default rainbow reps
    view.add_component(pdb_ref, ext="pdb", defaultRepresentation=False)
    view.add_component(pdb_aligned, ext="pdb", defaultRepresentation=False)

    view.clear_representations()

    # Reference (blue, translucent)
    view.add_cartoon(
        component=0,
        selection=selection,
        colorScheme="uniform",
        colorValue="blue",
        opacity=0.7
    )

    # Aligned (red, solid)
    view.add_cartoon(
        component=1,
        selection=selection,
        colorScheme="uniform",
        colorValue="red",
        opacity=1.0
    )

    view.center()
    return view


In [78]:
upload1 = widgets.FileUpload(accept=".pdb", multiple=False, description="Upload PDB 1")
upload2 = widgets.FileUpload(accept=".pdb", multiple=False, description="Upload PDB 2")

chain1 = widgets.Text(value="A", description="Chain 1")
chain2 = widgets.Text(value="A", description="Chain 2")

start1 = widgets.IntText(description="Start 1")
end1   = widgets.IntText(description="End 1")
start2 = widgets.IntText(description="Start 2")
end2   = widgets.IntText(description="End 2")

btn_range = widgets.Button(description="Auto-fill ranges")
btn_run = widgets.Button(description="Align + Visualize", button_style="primary")

out = widgets.Output()
workdir = tempfile.mkdtemp(prefix="tmalign_upload_tool_")


In [79]:
def autofill_ranges(_):
    out.clear_output()
    with out:
        try:
            pdb1 = save_upload(upload1, workdir)
            pdb2 = save_upload(upload2, workdir)

            s1, e1 = chain_range_from_pdb(pdb1, chain1.value)
            s2, e2 = chain_range_from_pdb(pdb2, chain2.value)

            start1.value, end1.value = s1, e1
            start2.value, end2.value = s2, e2

            print("Ranges auto-filled.")
        except Exception as e:
            print("ERROR:", e)

btn_range.on_click(autofill_ranges)


In [80]:
def run_align(_):
    out.clear_output()
    with out:
        try:
            pdb1 = save_upload(upload1, workdir)
            pdb2 = save_upload(upload2, workdir)

            seg1 = os.path.join(workdir, "seg1.pdb")
            seg2 = os.path.join(workdir, "seg2.pdb")

            parser = PDBParser(QUIET=True)
            io = PDBIO()

            io.set_structure(parser.get_structure("X", pdb1))
            io.save(seg1, select=ChainRangeSelect(chain1.value, start1.value, end1.value))

            io.set_structure(parser.get_structure("Y", pdb2))
            io.save(seg2, select=ChainRangeSelect(chain2.value, start2.value, end2.value))

            aligned_pdb, stdout = run_tmalign_write(
                seg1, seg2, out_dir=workdir, out_name="aligned"
            )

            print(stdout.splitlines()[0])
            display(visualize_ngl(seg2, aligned_pdb))


        except Exception as e:
            print("ERROR:", e)

btn_run.on_click(run_align)


## Download Structures in .pdb format from RCSB.org or predicted structures in .pdb format
1. Upload your structures one by one
2. Specify chains A or B etc..
3. Auto-fill ranges to fill start and end range of structures
4. if you want to align specific domains then fill in those ranges or leave it as it is!
5. Align+visualize

In [81]:
tmalign_ui = widgets.VBox([
    widgets.HBox([upload1, upload2]),
    widgets.HBox([chain1, start1, end1]),
    widgets.HBox([chain2, start2, end2]),
    widgets.HBox([btn_range, btn_run]),
    out,
])


In [82]:
import os, tempfile, traceback
import ipywidgets as w
from IPython.display import display, clear_output, IFrame
from IPython.display import HTML


from IPython.display import HTML

def display_scrollable_df(
    df,
    max_height="420px",
    max_width="100%"
):
    if df is None:
        return

    html_table = df.to_html(index=False, escape=False)

    css = f"""
    <style>
      .scroll-df-wrap {{
        width: 100%;
        max-width: {max_width};
        max-height: {max_height};
        overflow-x: auto;   /* ← horizontal scroll */
        overflow-y: auto;   /* ← vertical scroll */
        border: 1px solid #e0e0e0;
        border-radius: 6px;
        box-sizing: border-box;
      }}
      .scroll-df-wrap table {{
        min-width: 100%;    /* ← forces table to stay inside container */
        border-collapse: collapse;
        font-size: 13px;
      }}
      .scroll-df-wrap th,
      .scroll-df-wrap td {{
        padding: 6px 8px;
        border-bottom: 1px solid #eee;
        text-align: left;
        white-space: nowrap;  /* ← keeps rows single-line */
      }}
      .scroll-df-wrap thead th {{
        position: sticky;
        top: 0;
        background: #fafafa;
        z-index: 2;
        border-bottom: 1px solid #ddd;
      }}
    </style>
    """

    display(HTML(css + f"<div class='scroll-df-wrap'>{html_table}</div>"))



def _err(out: w.Output, e: Exception):
    with out:
        print("❌ Error:", e)
        traceback.print_exc()

def _need_acc(out: w.Output):
    if not state["acc"]:
        with out:
            print("Set a UniProt accession first (top bar).")
        return True
    return False

def download_alphafold_pdb(accession: str, out_dir: str) -> str:
    """Download AlphaFold PDB (model_v4) to out_dir and return file path."""
    import requests
    os.makedirs(out_dir, exist_ok=True)
    url = f"https://alphafold.ebi.ac.uk/files/AF-{accession}-F1-model_v6.pdb"
    fp = os.path.join(out_dir, f"AF-{accession}-F1-model_v6.pdb")
    r = requests.get(url, timeout=120)
    r.raise_for_status()
    with open(fp, "wb") as f:
        f.write(r.content)
    return fp

# ----------------------------
# Top bar: select protein
# ----------------------------
acc_input = w.Text(description="UniProt:", placeholder="e.g., P07949", layout=w.Layout(width="320px"))
btn_set = w.Button(description="Set protein", button_style="info")
lbl = w.HTML("<b>Current:</b> (not set)")
top_out = w.Output(layout={"border":"1px solid #ddd","padding":"6px"})

def on_set(_):
    with top_out:
        clear_output()
        acc = acc_input.value.strip()
        if not acc:
            print("Please enter a UniProt accession.")
            return
        state["acc"] = acc
        lbl.value = f"<b>Current:</b> {acc}"
        print(f"✅ Protein set: {acc}")
        print(f"Session workdir: {state['workdir']}")

btn_set.on_click(on_set)
top = w.VBox([w.HBox([acc_input, btn_set, lbl]), top_out])

# ----------------------------
# TAB 1: PTMs (Scop3P)
# ----------------------------
out_ptm = w.Output(layout={"border":"1px solid #ddd","padding":"6px"})
btn_fetch_ptm = w.Button(description="Fetch PTMs", button_style="warning")
btn_show_ptm = w.Button(description="Show table")

def fetch_ptm(_):
    try:
        with out_ptm:
            clear_output()
            if _need_acc(out_ptm): 
                return
            print("Fetching PTMs...")
        res = fetch_protein_modifications(state["acc"])
        tbl = get_modification_table(res.get("modifications", []))
        state["ptm_json"] = res
        state["ptm_table"] = tbl

        # Keep a global name for legacy functions that expect it (e.g., display_b2b_3D)
        globals()["modification_table"] = tbl

        with out_ptm:
            print("✅ Done.")
            display_scrollable_df(tbl, max_height="420px",max_width="95vw")

    except Exception as e:
        _err(out_ptm, e)

def show_ptm(_):
    with out_ptm:
        clear_output()
        if state["ptm_table"] is None:
            print("No PTM table yet. Click 'Fetch PTMs'.")
            return
        display_scrollable_df(state["ptm_table"], max_height="420px",max_width="95vw")


btn_fetch_ptm.on_click(fetch_ptm)
btn_show_ptm.on_click(show_ptm)

tab1 = w.VBox([w.HTML("<h3>Scop3P PTMs</h3>"), w.HBox([btn_fetch_ptm, btn_show_ptm]), out_ptm])

# ----------------------------
# TAB 2: Variants (UniProt/EBI API)
# ----------------------------
out_var = w.Output(layout={"border":"1px solid #ddd","padding":"6px"})
btn_fetch_var = w.Button(description="Fetch disease-associated variants", button_style="warning")

def fetch_var(_):
    try:
        with out_var:
            clear_output()
            if _need_acc(out_var): 
                return
            print("Fetching variants...")
        df = fetch_uniprot_variants_disease(state["acc"])
        state["variants_df"] = df
        with out_var:
            print("✅ Done.")
            display_scrollable_df(df, max_height="420px",max_width="95vw")
    except Exception as e:
        _err(out_var, e)

btn_fetch_var.on_click(fetch_var)
tab2 = w.VBox([w.HTML("<h3>Disease-associated variants</h3>"), btn_fetch_var, out_var])

# TAB 3: 3D structure viewer (PDB + AlphaFold)  [NGLVIEW ONLY]
# ----------------------------
import os
import ipywidgets as w
from IPython.display import display, clear_output
import nglview as nv
import requests

out_3d = w.Output(layout={"border":"1px solid #ddd","padding":"8px"})

structure_source = w.ToggleButtons(
    options=[("PDB", "pdb"), ("AlphaFold", "af")],
    value="pdb",
    description="Source:"
)

pdb_input   = w.Text(description="PDB:", placeholder="e.g., 2IVT", layout=w.Layout(width="240px"))
chain_input = w.Text(description="Chain:", placeholder="A (optional)", layout=w.Layout(width="200px"))

btn_fetch_af = w.Button(description="Fetch AlphaFold", button_style="warning")
btn_show_3d  = w.Button(description="Show 3D", button_style="success")

# --- Styling + PTM coloring ---
BASE_GREY = "white"
# Your py3Dmol example used different colors; you asked for: SER red, THR green, TYR orange
PTM_COLOR = {"SER": "red", "THR": "green", "TYR": "orange"}

def _download_pdb_to_workdir(pdb_id: str, outdir: str) -> str:
    """
    Download PDB from RCSB to workdir as .pdb (only if not already present).
    """
    pdb_id = pdb_id.strip().lower()
    os.makedirs(outdir, exist_ok=True)
    outpath = os.path.join(outdir, f"{pdb_id}.pdb")

    if os.path.exists(outpath) and os.path.getsize(outpath) > 0:
        return outpath

    url = f"https://files.rcsb.org/download/{pdb_id.upper()}.pdb"
    r = requests.get(url, timeout=30)
    r.raise_for_status()

    with open(outpath, "wb") as f:
        f.write(r.content)
    return outpath

def _build_nglview_from_pdbfile(pdb_path: str):
    v = nv.show_file(pdb_path)
    v.camera = "orthographic"

    # Turn on NGL's built-in hover tooltip (closest equivalent to py3Dmol hover labels)
    try:
        v._remote_call("setParameters", target="stage", kwargs={"tooltip": True})
    except Exception:
        pass

    return v

def _style_base_ngl(view, chain=None):
    """
    Grey cartoon + grey surface (opacity 0.5).
    If chain is provided, restrict to that chain.
    """
    try:
        view.clear_representations()
    except Exception:
        pass

    chain = (chain or "").strip()
    if chain:
        chain = chain[0].upper()
        sel = f"protein and chain {chain}"
    else:
        sel = "protein"

    view.add_representation("cartoon", selection=sel, color="grey")
    # view.add_representation("surface", selection=sel, color="grey", opacity=0.5)




def _collect_ptms_from_table(ptm_table):
    """
    Returns list of tuples: (resi_int, residue_name_string)
    Expects columns like your py3Dmol function: 'position' and 'residue'
    but includes mild fallbacks.
    """
    if ptm_table is None:
        return []

    # Identify columns
    pos_col = None
    for c in ["position", "modpos", "Position", "MODPOS"]:
        if c in ptm_table.columns:
            pos_col = c
            break

    res_col = None
    for c in ["residue", "modres", "Residue", "MODRES"]:
        if c in ptm_table.columns:
            res_col = c
            break

    if pos_col is None or res_col is None:
        return []

    out = []
    for _, row in ptm_table[[pos_col, res_col]].dropna().iterrows():
        try:
            pos = int(row[pos_col])
        except Exception:
            continue
        res = str(row[res_col]).strip().upper()
        out.append((pos, res))
    return out

def _highlight_ptms_ngl(view, ptm_table, chain=None):
    """
    Robust NGL selection:
      - single residue on chain: "123:A"
      - single residue (no chain): "123"
    """
    ptms = _collect_ptms_from_table(ptm_table)
    if not ptms:
        return

    chain = (chain or "").strip()
    if chain:
        chain = chain[0].upper()

    for pos, res in ptms:
        color = PTM_COLOR.get(res, "#7B241C")

        # NGL-safe selection
        if chain:
            sel = f"{int(pos)}:{chain}"   # e.g. "123:A"
        else:
            sel = f"{int(pos)}"           # AF branch usually OK without chain

        # Only the selected residue gets these reps
        view.add_representation("ball+stick", selection=sel, color=color)
        view.add_representation("spacefill", selection=sel, color=color, radius=0.9)



def on_fetch_af(_):
    try:
        with out_3d:
            clear_output()
            if not state.get("acc"):
                print("Set a UniProt accession first (header).")
                return

            print("Downloading AlphaFold model...")
            af_path = download_alphafold_pdb(state["acc"], out_dir=state["workdir"])
            state["af_path"] = af_path
            print("✅ Saved:", af_path)

    except Exception as e:
        with out_3d:
            print("❌ Failed to fetch AlphaFold:", e)

def on_show_3d(_):
    try:
        with out_3d:
            clear_output()

            ptm_table = state.get("ptm_table")

            # -------- AlphaFold branch --------
            if structure_source.value == "af":
                af_path = state.get("af_path")
                if not af_path or not os.path.exists(af_path):
                    print("No AlphaFold model found. Click 'Fetch AlphaFold' first.")
                    return

                v = _build_nglview_from_pdbfile(af_path)
                _style_base_ngl(v)
                _highlight_ptms_ngl(v, ptm_table, chain='A')  # AF usually single chain
                display(v)
                return

            # -------- PDB branch (NGLView, NOT py3Dmol) --------
            pdb = pdb_input.value.strip()
            if not pdb:
                print("Enter a PDB ID.")
                return

            ch = chain_input.value.strip() or None
            if ch:
                ch = ch[0].upper()


            pdb_path = _download_pdb_to_workdir(pdb, state["workdir"])
            v = _build_nglview_from_pdbfile(pdb_path)
            _style_base_ngl(v,chain=ch)
            _highlight_ptms_ngl(v, ptm_table, chain=ch)

            display(v)

    except Exception as e:
        with out_3d:
            print("❌ Error:", e)

btn_fetch_af.on_click(on_fetch_af)
btn_show_3d.on_click(on_show_3d)

tab3 = w.VBox([
    w.HTML("<h3>3D Structure Viewer</h3>"),
    w.HBox([structure_source, btn_fetch_af, btn_show_3d]),
    w.HBox([pdb_input, chain_input]),
    out_3d
])




# ----------------------------
# TAB 4: Bio2Byte predictions (on-demand)
# ----------------------------
out_b2b = w.Output(layout={"border":"1px solid #ddd","padding":"6px"})
btn_fetch_seq = w.Button(description="Fetch sequence", button_style="warning")
btn_run_b2b = w.Button(description="Run predictions", button_style="danger")
btn_show_b2b = w.Button(description="Show prediction table", button_style="success")
btn_show_b2b3d = w.Button(description="3D panel (Bio2Byte colors)", button_style="")

b2b_metric = w.Dropdown(
    description="Color by:",
    options=[],  # filled after predictions
    layout=w.Layout(width="320px")
)
b2b_chain = w.Text(description="Chain:", placeholder="A (optional)", layout=w.Layout(width="200px"))

viewer_mode = w.ToggleButtons(
    options=[("Single (NGL)", "ngl"), ("4-panel (py3Dmol)", "py3dmol")],
    value="ngl",
    description="Viewer:",
    layout=w.Layout(margin="0 0 0 10px")
)

import colorsys
import py3Dmol

def pseudocolor(minval, maxval, val):
    """Map value in [minval,maxval] to green→yellow→red hex (HSV)."""
    minval, maxval = float(minval), float(maxval)
    if maxval == minval:
        h = 120.0
    else:
        h = (float(maxval - val) / (maxval - minval)) * 120.0  # 120=green, 0=red
    r, g, b = colorsys.hsv_to_rgb(h/360.0, 1.0, 1.0)
    rgb = tuple(int(255*x) for x in (r, g, b))
    return "0x%02x%02x%02x" % rgb

def remap_b2b_colors(df):
    """
    Build per-residue color maps for Bio2Byte columns:
      - backbone
      - disoMine
      - earlyFolding
    Assumes df rows correspond to positions 1..N (Bio2Byte output aligned to sequence).
    """
    BDcolor, EFcolor, DOcolor = {}, {}, {}
    seqpos = 0

    # Use column names exactly as in your non-Voilà version
    min_BD, max_BD = float(df["backbone"].min()), float(df["backbone"].max())
    min_DO, max_DO = float(df["disoMine"].min()), float(df["disoMine"].max())
    min_EF, max_EF = float(df["earlyFolding"].min()), float(df["earlyFolding"].max())

    for _, row in df.iterrows():
        seqpos += 1
        BDcolor[seqpos] = pseudocolor(min_BD, max_BD, float(row["backbone"]))
        DOcolor[seqpos] = pseudocolor(min_DO, max_DO, float(row["disoMine"]))       # ✅ fixed
        EFcolor[seqpos] = pseudocolor(min_EF, max_EF, float(row["earlyFolding"]))
    return BDcolor, EFcolor, DOcolor

def display_b2b_4panel_py3dmol(dynamic_properties_df, pdb_path: str, ptm_positions=None):
    """
    2×2 py3Dmol grid:
      (0,0) pLDDT from AF b-factor (0..100)
      (0,1) backbone (resi-mapped colors)
      (1,0) disoMine (resi-mapped colors)
      (1,1) earlyFolding (resi-mapped colors)
    PTMs as VDW surface + faint spheres; hover labels enabled.
    """
    # Build color maps (resi -> hex)
    BDcolor, EFcolor, DOcolor = remap_b2b_colors(dynamic_properties_df)

    # Read PDB content once
    with open(pdb_path, "r") as f:
        pdb_txt = f.read()

    view = py3Dmol.view(viewergrid=(2, 2), linked=True, width=950, height=740)
    view.addModel(pdb_txt, "pdb")

    # Panel (0,0): pLDDT from b-factor
    view.setStyle(
        {},
        {"cartoon": {"colorscheme": {"prop": "b", "gradient": "rwb", "min": 0.0, "max": 100.0}}},
        viewer=(0, 0),
    )

    # Panel (0,1): backbone
    view.setStyle({}, {"cartoon": {"colorscheme": {"prop": "resi", "map": BDcolor}}}, viewer=(0, 1))

    # Panel (1,0): disorder
    view.setStyle({}, {"cartoon": {"colorscheme": {"prop": "resi", "map": DOcolor}}}, viewer=(1, 0))

    # Panel (1,1): early folding
    view.setStyle({}, {"cartoon": {"colorscheme": {"prop": "resi", "map": EFcolor}}}, viewer=(1, 1))

    # PTM overlay as surface + faint spheres (pickable)
    if ptm_positions:
        for mod in sorted(set(int(x) for x in ptm_positions)):
            sel = {"resi": str(mod)}

            view.addSurface(py3Dmol.VDW, {"opacity": 1.0}, sel, viewer=(0, 0))
            view.addSurface(py3Dmol.VDW, {"opacity": 1.0, "color": BDcolor.get(mod, "0xff0000")}, sel, viewer=(0, 1))
            view.addSurface(py3Dmol.VDW, {"opacity": 1.0, "color": DOcolor.get(mod, "0xff0000")}, sel, viewer=(1, 0))
            view.addSurface(py3Dmol.VDW, {"opacity": 1.0, "color": EFcolor.get(mod, "0xff0000")}, sel, viewer=(1, 1))

            for panel in [(0,0), (0,1), (1,0), (1,1)]:
                view.addStyle(sel, {"sphere": {"radius": 0.8, "opacity": 0.15}}, viewer=panel)

    # Background + hover labels (per panel)
    for panel in [(0,0), (0,1), (1,0), (1,1)]:
        view.setBackgroundColor("white", viewer=panel)

        view.setHoverable(
            {}, True,
            """
            function(atom, viewer, event, container) {
                if (!atom.label) {
                    atom.label = viewer.addLabel(
                        atom.resn + " " + atom.resi + (atom.chain ? (" : " + atom.chain) : ""),
                        {position: atom, backgroundColor: 'mintcream', fontColor:'black'}
                    );
                }
            }
            """,
            """
            function(atom, viewer) {
                if (atom.label) {
                    viewer.removeLabel(atom.label);
                    delete atom.label;
                }
            }
            """,
            viewer=panel
        )

    view.zoomTo()
    view.render()
    return view


def fetch_seq(_):
    try:
        with out_b2b:
            clear_output()
            if _need_acc(out_b2b): 
                return
            print("Fetching sequence...")
        _pid, seq = fetch_sequence_aminoacids(state["acc"])
        state["sequence"] = seq
        with out_b2b:
            print(f"✅ Sequence length: {len(seq)} aa")
    except Exception as e:
        _err(out_b2b, e)

def run_b2b(_):
    try:
        if not state["sequence"]:
            with out_b2b:
                clear_output()
                print("Fetch sequence first.")
            return
        btn_run_b2b.disabled = True
        with out_b2b:
            clear_output()
            print("Running Bio2Byte predictions (this can take a bit)...")
        pred = predict_biophysical_features(state["acc"], state["sequence"])
        state["bio2byte_raw"] = pred
        prot = pred.get("proteins", {}).get(state["acc"], {})
        import pandas as pd
        dyn = pd.DataFrame(prot)
        # pick numeric columns only
        num_cols = [c for c in dyn.columns if dyn[c].dtype.kind in "if"]
        b2b_metric.options = num_cols
        if num_cols:
            b2b_metric.value = num_cols[0]

        state["dynamic_properties"] = dyn
        # globals()["dynamic_properties"] = dyn  # for legacy cells if needed
        with out_b2b:
            print("✅ Done.")
            display(dyn.head())
    except Exception as e:
        _err(out_b2b, e)
    finally:
        btn_run_b2b.disabled = False

def show_b2b(_):
    with out_b2b:
        clear_output()
        if state["dynamic_properties"] is None:
            print("Run predictions first.")
            return
        display_scrollable_df(state["dynamic_properties"], max_height="420px",max_width="95vw")

def _pdb_with_bfactor_from_df(pdb_in: str, df, value_col: str, out_dir: str, chain: str = None) -> str:
    """
    Write a copy of pdb_in where B-factor is set per residue using df[value_col].
    Assumes df rows correspond to UniProt positions 1..N (Bio2Byte output).
    If chain is provided, only that chain gets updated; otherwise all chains get updated.
    """
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"b2b_{value_col}.pdb")

    # Bio2Byte df typically aligns to sequence positions 1..len(seq)
    # We'll map resSeq -> value using 1-based indexing.
    values = df[value_col].tolist()
    pos_to_val = {i + 1: float(v) for i, v in enumerate(values) if v is not None and v == v}

    def _set_b(line, b):
        # PDB B-factor columns are 61-66 (1-based), i.e. line[60:66] in 0-based
        b_str = f"{b:6.2f}"
        return line[:60] + b_str + line[66:]

    with open(pdb_in, "r") as fin, open(out_path, "w") as fout:
        for line in fin:
            if not (line.startswith("ATOM") or line.startswith("HETATM")):
                fout.write(line)
                continue

            # chain ID is column 22 (1-based) => line[21]
            ch = line[21].strip()
            if chain and ch and ch != chain:
                fout.write(line)
                continue

            # residue sequence number columns 23-26 (1-based) => line[22:26]
            try:
                resseq = int(line[22:26].strip())
            except:
                fout.write(line)
                continue

            # Only update if we have a value for this position
            if resseq in pos_to_val:
                fout.write(_set_b(line, pos_to_val[resseq]))
            else:
                fout.write(line)

    return out_path


def display_b2b_3D_ngl(dyn_df, pdb_path: str, value_col: str, chain: str = None):
    out_pdb = _pdb_with_bfactor_from_df(
        pdb_in=pdb_path,
        df=dyn_df,
        value_col=value_col,
        out_dir=state["workdir"],
        chain=chain
    )

    v = nv.show_file(out_pdb)
    v.clear_representations()
    
    # Base Bio2Byte coloring
    v.add_representation("cartoon", selection="protein", color_scheme="bfactor")
    
    if state.get("ptm_table") is not None and "position" in state["ptm_table"].columns:
        ptm_pos = state["ptm_table"]["position"].dropna().astype(int).tolist()
    
        chain = (chain or "").strip()
        if chain:
            chain = chain[0].upper()
    
        for pos in ptm_pos:
            if chain:
                sel = f"{int(pos)}:{chain}"
            else:
                sel = f"{int(pos)}"
    
            v.add_representation("ball+stick", selection=sel, color_scheme="bfactor",radius=1.1)



    # Auto-fit / zoom (version-safe)
    if hasattr(v, "center"):
        v.center()
    else:
        v._remote_call("autoView", target="stage")

    return v


def show_b2b3d(_):
    try:
        with out_b2b:
            clear_output()

            if state["dynamic_properties"] is None:
                print("Run predictions first.")
                return
            if not state.get("af_path"):
                print("Fetch AlphaFold model first (Tab 3).")
                return

            # ---- NGL single-panel ----
            if viewer_mode.value == "ngl":
                if not b2b_metric.value:
                    print("Pick a metric.")
                    return
                ch = (b2b_chain.value or "").strip() or None
                v = display_b2b_3D_ngl(
                    state["dynamic_properties"],
                    pdb_path=state["af_path"],
                    value_col=b2b_metric.value,
                    chain=ch
                )
                display(v)
                return  # ✅ important

            # ---- py3Dmol 4-panel ----
            if viewer_mode.value == "py3dmol":
                ptm_pos = None
                if state.get("ptm_table") is not None and "position" in state["ptm_table"].columns:
                    ptm_pos = state["ptm_table"]["position"].dropna().astype(int).tolist()

                v = display_b2b_4panel_py3dmol(
                    state["dynamic_properties"],
                    pdb_path=state["af_path"],
                    ptm_positions=ptm_pos
                )
                display(v)
                return

            print("Unknown viewer mode:", viewer_mode.value)

    except Exception as e:
        _err(out_b2b, e)

btn_fetch_seq.on_click(fetch_seq)
btn_run_b2b.on_click(run_b2b)
btn_show_b2b.on_click(show_b2b)
btn_show_b2b3d.on_click(show_b2b3d)

tab4 = w.VBox([
    w.HTML("<h3>Bio2Byte biophysical predictions</h3>"),
    w.HBox([btn_fetch_seq, btn_run_b2b, btn_show_b2b, btn_show_b2b3d]),
    w.HBox([viewer_mode, b2b_metric, b2b_chain]),
    out_b2b
])



# ----------------------------
# TAB 5: RIN (Residue Interaction Network)
# ----------------------------
out_rin = w.Output(layout={"border":"1px solid #ddd","padding":"6px"})
btn_dl_af = w.Button(description="Download AlphaFold PDB", button_style="warning")
upload_pdb = w.FileUpload(accept=".pdb", multiple=False, description="Or upload PDB")
chain_rin = w.Text(description="Chain:", placeholder="A", layout=w.Layout(width="180px"))
cutoff = w.FloatSlider(description="Cutoff Å:", min=4.0, max=12.0, step=0.5, value=8.0, readout=True)
btn_build_rin = w.Button(description="Build RIN", button_style="danger")
btn_show_rin = w.Button(description="Show RIN", button_style="success")

import os

def _save_upload_to_path(upl: w.FileUpload, out_dir: str) -> str:
    if not upl.value:
        raise ValueError("No PDB uploaded")

    # ipywidgets can provide:
    # - dict: {filename: {"content":..., "metadata":...}}
    # - list/tuple: [{"content":..., "name":...}, ...]
    v = upl.value

    if isinstance(v, dict):
        # key is usually the filename
        fname, item = next(iter(v.items()))
        name = (
            item.get("metadata", {}).get("name")
            or item.get("name")
            or fname
            or "uploaded.pdb"
        )
        content = item.get("content", b"")
    elif isinstance(v, (list, tuple)):
        item = v[0]
        name = item.get("metadata", {}).get("name") or item.get("name") or "uploaded.pdb"
        content = item.get("content", b"")
    else:
        raise TypeError(f"Unexpected FileUpload.value type: {type(v)}")

    # content might be memoryview/bytearray
    if isinstance(content, memoryview):
        content = content.tobytes()
    elif isinstance(content, bytearray):
        content = bytes(content)

    os.makedirs(out_dir, exist_ok=True)
    fp = os.path.join(out_dir, name)
    with open(fp, "wb") as f:
        f.write(content)

    # sanity check
    if os.path.getsize(fp) < 100:
        raise ValueError(f"Uploaded file looks too small ({os.path.getsize(fp)} bytes). Not a valid PDB?")

    return fp


def dl_af(_):
    try:
        with out_rin:
            clear_output()
            if _need_acc(out_rin): 
                return
            print("Downloading AlphaFold PDB...")
        fp = download_alphafold_pdb(state["acc"], state["workdir"])
        state["rin_pdb_path"] = fp
        with out_rin:
            print(f"✅ Saved: {fp}")
    except Exception as e:
        _err(out_rin, e)

def build_rin(_):
    try:
        with out_rin:
            clear_output()
            pdb_path = None
            if upload_pdb.value:
                pdb_path = _save_upload_to_path(upload_pdb, state["workdir"])
                
                print("Using pdb_path:", pdb_path)
                print("File size (bytes):", os.path.getsize(pdb_path))

            elif state.get("rin_pdb_path"):
                pdb_path = state["rin_pdb_path"]
            else:
                print("Upload a PDB or download AlphaFold first.")
                return

            ch = (chain_rin.value or "A").strip()
            print(f"Building RIN from: {os.path.basename(pdb_path)} (chain {ch}) cutoff {cutoff.value} Å ...")

        tmp = build_geometry_graph_from_pdb(pdb_path, chain=ch, cutoff=float(cutoff.value))
            

        G = tmp[0] if isinstance(tmp, tuple) else tmp
        print("Chain used for RIN:", G.graph.get("chain"))
        ptm_pos = None
        mut_pos = None
        if state.get("ptm_table") is not None and "position" in state["ptm_table"].columns:
            ptm_pos = list(set(state["ptm_table"]["position"].dropna().astype(int).tolist()))
        if state.get("variants_df") is not None and "position" in state["variants_df"].columns:
            mut_pos = list(set(state["variants_df"]["position"].dropna().astype(int).tolist()))

        out_html = os.path.join(state["workdir"], f"rin_pyvis_{state['acc'] or 'session'}.html")
        html_path = nx_rin_to_pyvis_default(G, ptm_positions=ptm_pos, mutation_positions=mut_pos, out_html=out_html)
        state["rin_html"] = html_path

        with out_rin:
            print("✅ RIN built.")
            print("Click 'Show RIN' to open the interactive network.")

    except Exception as e:
        _err(out_rin, e)


from IPython.display import HTML

def show_rin(_):
    with out_rin:
        clear_output()
        p = state.get("rin_html")
        if not p or not os.path.exists(p):
            print("No RIN HTML yet. Build it first.")
            return
        with open(p, "r", encoding="utf-8") as f:
            html = f.read()
        display(HTML(html))



btn_dl_af.on_click(dl_af)
btn_build_rin.on_click(build_rin)
btn_show_rin.on_click(show_rin)

tab5 = w.VBox([
    w.HTML("<h3>Residue Interaction Network (PyVis)</h3>"),
    w.HBox([btn_dl_af, upload_pdb]),
    w.HBox([chain_rin, btn_build_rin, btn_show_rin]), ## cutoff for cutoff slider
    out_rin
])

# ----------------------------
# TAB 6: TM-align tool (existing UI)
# ----------------------------
out_tm = w.Output(layout={"border":"1px solid #ddd","padding":"6px"})
tab6 = w.VBox([
    w.HTML("<h3>TM-align (upload two structures, optional residue range)</h3>"),
    w.HTML("<p><b>Note:</b> TM-align must be installed in the container/server and available on PATH as <code>TM-align</code>.</p>"),
    tmalign_ui,
])

# ----------------------------
# Assemble tabs
# ----------------------------
tabs = w.Tab(children=[tab1, tab2, tab3, tab4, tab5, tab6])
titles = ["1) PTMs", "2) Variants", "3) 3D Viewer", "4) Bio2Byte", "5) RIN", "6) TM-align"]
for i,t in enumerate(titles):
    tabs.set_title(i, t)

display(w.VBox([top, tabs]))
